In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import collections
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib.image as mpimg
from tqdm import tqdm
from typing import List, Tuple, Dict, Any
import gc

# Get statistics over all matching pipelines

In [2]:

#####################
### FUNCTIONS
#####################
def score_function(group):
    """The scorefunction S(Theta)"""
    # Belohne Qualität / Smooth
    # lambda  * 0.5 * (t1 + t2)
    # t1 = tanh(0.5 * tanh(A) + tanh(B*B))
    # t2 = tanh(0.5 * tanh(C) + tanh(D*D))
    
    # Get lambda factor. Note: lambda is reserved word in python
    lambdaa = group[['lambda']].values.reshape(-1, 1)
    
    A = group.matchability.values.reshape(-1, 1)
    B = group.accuracy_matches.values.reshape(-1, 1)
    C = group.inlier_ratio.values.reshape(-1, 1)
    D = group.accuracy_inliers.values.reshape(-1, 1)
    
    t1 = np.tanh(0.5 * np.tanh(A) + np.tanh(B*B))
    t2 = np.tanh(0.5 * np.tanh(C) + np.tanh(D*D))
    
    values = np.hstack([t1, t2])
    values *= lambdaa
    
    # Build mean for each column
    mean_values = np.mean(values, axis=0)

    # Build mean of means to get score
    score = np.mean(mean_values)
    
    return score

def create_scoremap(
    df:pd.DataFrame,
    desc2row:Dict,
    det2col:Dict,
    filters:Dict={}) -> np.array:
    """Creates a single scoremap for ALL triplets 
    (kp_thresh, descriptor_name, detector_name)"""
    
    descriptor_names = sorted(df.descriptor_name.unique())
    detector_names = sorted(df.detector_name.unique())
    scoremap = np.full((len(descriptor_names), len(detector_names)), np.nan)
    
    # Filter df for each key-value pair in filters
    _df = df
    for k, v in filters.items():
        _df = _df[df[k] == v]

    # Group by descriptors and detectors
    _g = _df.groupby(['descriptor_name', 'detector_name'])

    for gname, group in _g:
        i = desc2row[gname[0]]
        j = det2col[gname[1]]
        
        score = score_function(group)

        scoremap[i][j] = score
    
    return scoremap


def find_best_matching_method_by_avg_score(
    evaluations:List[str],
    eval_dict:Dict,
    desc2row:Dict,
    det2col:Dict,
    filters:Dict={}) -> Tuple[np.array, np.array, np.array]:
    """Returns labels and avg score for each matching method in
    descending order as well as their sorting indices."""
    scoremaps = []
    labels = []
    for evaluation in evaluations:
        df = get_dataframe(evaluation, eval_dict)

        scoremaps.append(create_scoremap(df, desc2row, det2col, filters=filters))
        labels.append(eval_dict[evaluation]['label'])

    # Convert to np.array
    scoremaps = np.array(scoremaps)
    labels = np.array(labels)

    # Build average score for an evaluation
    avg_scores = np.array([np.nanmean(x) for x in scoremaps])

    # sort by avg_scores
    idx_descending = avg_scores.argsort()[::-1]
    avg_scores = avg_scores[idx_descending]
    labels = labels[idx_descending]
    
    return labels, avg_scores, idx_descending

def get_dataframe(evaluation_method:str, eval_dict:Dict) -> pd.DataFrame:
    file_name = eval_dict[evaluation_method]['file_name'] + '.csv'
    
    # Load data
    df = pd.read_csv(os.path.join(data_dir, file_name), sep=',', comment='#')
    
    return df


#####################
### SETTINGS
#####################
desc2row = {
    'doap': 0,
    'lift': 1,
    'sift': 2,
    'superpoint': 3,
    'tfeat': 4
}

det2col = {
    'lift': 0,
    'sift': 1,
    'superpoint': 2,
    'tcovdet': 3,
    'tilde': 4
}


id2name = {
    'sift': 'SIFT',
    'lift': 'LIFT',
    'tilde': 'TILDE',
    'superpoint': 'SuperPoint',
    'tcovdet': 'TCovDet',
    'tfeat': 'TFeat',
    'doap': 'DOAP',
    'v_set_01': 'Set 1',
    'v_set_02': 'Set 2',
    'v_set_03': 'Set 3',
    'v_set_04': 'Set 4',
    'v_set_05': 'Set 5',
    'v_set_06': 'Set 6',
    'v_set_07': 'Set 7',
    'v_set_08': 'Set 8',
    'v_set_09': 'Set 9',
    'v_set_10': 'Set 10',
    'v_set_11': 'Set 11',
    'v_set_12': 'Set 12',
    'v_set_13': 'Set 13'
}

color_scheme = {
  'sift': '#1f77b4',
  'lift': '#ff7f0e',
  'tilde': '#2ca02c',
  'superpoint': '#d62728',
  'tcovdet': '#9467bd',
  'tfeat': '#2ca02c',
  'doap': '#9467bd'
}

keypoint_threshold_colors = ['skyblue', 'purple', 'crimson']

marker_scheme = {
  'sift': 'o',
  'lift': 'x',
  'tilde': 'd',
  'superpoint': '^',
  'tcovdet': 'h',
  'tfeat': '*',
  'doap': 'p'
}

line_styles = ['-', '-.', ':', '--']

eval_dict = {
    'nn2way': {
        'file_name': 'descriptor_matching_eisert_nn2way',
        'label': 'NN2WAY'
    },
    'bf_ratio': {
        'file_name': 'descriptor_matching_eisert_bf_ratio',
        'label': 'BF + Ratio'
    },
    'bf': {
        'file_name': 'descriptor_matching_eisert_bf',
        'label': 'BF'
    },
    'flann': {
        'file_name': 'descriptor_matching_eisert_flann',
        'label': 'FLANN'
    },
    'flann_ratio': {
        'file_name': 'descriptor_matching_eisert_flann_ratio',
        'label': 'FLANN + Ratio'
    },
    'nn2way_no_normalization': {
        'file_name': 'descriptor_matching_eisert_nn2way_no_normalization',
        'label': 'NN2WAY\n(no normalization)'
    },
    'bf_ratio_no_normalization': {
        'file_name': 'descriptor_matching_eisert_bf_ratio_no_normalization',
        'label': 'BF + Ratio\n(no normalization)'
    },
    'bf_no_normalization': {
        'file_name': 'descriptor_matching_eisert_bf_no_normalization',
        'label': 'BF\n(no normalization)'
    },
    'flann_no_normalization': {
        'file_name': 'descriptor_matching_eisert_flann_no_normalization',
        'label': 'FLANN\n(no normalization)'
    },
    'flann_ratio_no_normalization': {
        'file_name': 'descriptor_matching_eisert_flann_ratio_no_normalization',
        'label': 'FLANN + Ratio\n(no normalization)'
    },
}


# nn2way | bf | bf_ratio | flann | flann_ratio | nn2way_no_normalization | bf_no_normalization | 
# bf_ratio_no_normalization | flann_no_normalization | flann_ratio_no_normalization

evaluations = ['nn2way', 'bf', 'bf_ratio', 'flann', 'flann_ratio', 'nn2way_no_normalization', 
               'bf_no_normalization',  'bf_ratio_no_normalization', 'flann_no_normalization', 
               'flann_ratio_no_normalization']


data_dir = '/home/mizzade/Workspace/diplom/outputs/eval_matching_pipeline'
output_dir = '/home/mizzade/Workspace/diplom/outputs/eval_matching_pipeline/'
fout_name = 'comparison.csv'

#####################
### MAIN
#####################
save_outputs = False

# Build df from all Evaluations
df_list = []

for evaluation in evaluations:
    
    # Load data
    df = get_dataframe(evaluation, eval_dict)
    df_list.append(df)


df = pd.concat(df_list, ignore_index=True)

matching_methods = sorted(df.matching_method.unique())
descriptor_names = sorted(df.descriptor_name.unique())
detector_names = sorted(df.detector_name.unique())
set_names = sorted(df.set_name.unique())
kpts_thresholds = sorted(df.kpts_threshold.unique())

Finde das Verfahren mit den im Durchschnitt besten Werten für alle Detektor-Deskriptor Kombinationen

In [32]:
df.shape, df.columns

((45045, 23),
 Index(['collection_name', 'set_name', 'kpts_threshold', 'descriptor_name',
        'detector_name', 'matching_method', 'desc_distance_threshold',
        'ransac_threshold', 'ransac_confidence', 'num_kpts_i', 'num_kpts_j',
        'max_num_matches', 'num_matches', 'matchability', 'accuracy_matches',
        'mse_matching', 'max_num_inliers', 'num_inliers', 'inlier_ratio',
        'avg_distance', 'mse_estimation', 'lambda', 'accuracy_inliers'],
       dtype='object'))

In [3]:
def custom_score_function(row):
    """ Gets a pandas dataframe row, evaluates score"""
    
    R1 =  0.5 * np.tanh(row['matchability'])
    A1 = np.tanh(row['accuracy_matches'] ** 2)
    
    R2 = 0.5 * np.tanh(row['inlier_ratio'])
    A2 = np.tanh(row['accuracy_inliers'])
    
    score = 0.5 * row['lambda'] * (np.tanh(R1 + A1) + np.tanh(R2 + A2))
    
    return score

In [25]:
# Um die beste Matching Strategie zuer erhalten:
# Errechne die Punktzahlen für jedes Bilderset UNABHÄNGIG VON 
# - DETEKTOR
# - DESKRIPTOR
# - KPTHRESH

# 1. Group Dataframe only by matching method
g_by_mm = df.groupby(['matching_method'])

# 2. Score each row in dataframe
scores_by_mm = g_by_mm.apply(custom_score_function) # Returns pandas series


# # 3. Compute mean, min, max, and std of scores
mm_mean = scores_by_mm.groupby('matching_method').mean().rename('mean')
mm_std = scores_by_mm.groupby('matching_method').std().rename('std')
mm_min = scores_by_mm.groupby('matching_method').min().rename('min')
mm_max = scores_by_mm.groupby('matching_method').max().rename('max')


# # 4. Create dataframe for it
df_by_mm = pd.concat([mm_mean, mm_std, mm_min, mm_max], axis=1)

# 5. Sort by mean score
df_by_mm.sort_values(by=['mean'], ascending=False, inplace=True)
df_by_mm.round(3)

,mean,std,min,max
matching_method,,,,
flann,0.401,0.188,0.0,0.700
bf,0.399,0.193,0.0,0.707
nn2way,0.392,0.196,0.0,0.711
bf_ratio,0.348,0.261,0.0,0.714
flann_ratio,0.347,0.259,0.0,0.713
nn2way_no_normalization,0.275,0.118,0.0,0.637
bf_no_normalization,0.218,0.200,0.0,0.699
flann_no_normalization,0.215,0.200,0.0,0.700
flann_ratio_no_normalization,0.147,0.207,0.0,0.719


In [16]:
# Die beste Matching Methode lautet bestMM
bestMM = df_by_mm.iloc[0].name
print('Die beste Matching Strategie ist {}.'.format(bestMM))

# Betrachte im Folgenden nur Werte fuer die beste Matching Methode betsMM
df_bestMM = df[df.matching_method == bestMM]

Die beste Matching Strategie ist flann.


Finde nun die Durchschnittswertungen aller Detektor/Deskriptor Kombinationen unabhängig von der Anzahl geforderter Keypoints $\tau_{kp}$ bei der Verwendung von FLANN als Matching Verfahren.

In [84]:
# Um die beste Kombination aus Detektor und Deskriptor unabhängig von
# KP_Thresh zu erhalten, errechne die Durchschnittliche Punktzahl 
# für die bestMM für alle Kombinationen  über alle KP_THRESH

# 1. Group Dataframe only by by combinations of detector and descriptor
#g_by_ddcombi = df.groupby(['detector_name', 'descriptor_name'])
g_by_ddcombi = df_bestMM.groupby(['detector_name', 'descriptor_name'])

# 2. Score each row in dataframe
scores_by_ddcombi = g_by_ddcombi.apply(custom_score_function) # Returns pandas series

# 3. Compute mean, min, max, and std of scores
ddcombi_mean = scores_by_ddcombi.groupby(['detector_name', 'descriptor_name']).mean().rename('mean')
ddcombi_std = scores_by_ddcombi.groupby(['detector_name', 'descriptor_name']).std().rename('std')
ddcombi_min = scores_by_ddcombi.groupby(['detector_name', 'descriptor_name']).min().rename('min')
ddcombi_max = scores_by_ddcombi.groupby(['detector_name', 'descriptor_name']).max().rename('max')

# # 4. Create dataframe for it
df_by_ddcombi = pd.concat([ddcombi_mean, ddcombi_std, ddcombi_min, ddcombi_max], axis=1)
df_by_ddcombi

# 5. Sort by mean score
df_by_ddcombi.sort_values(by=['mean'], ascending=False, inplace=True)
df_by_ddcombi.round(3)

mean    std    min    max
detector_name descriptor_name                            
superpoint    lift             0.597  0.018  0.558  0.623
lift          lift             0.576  0.107  0.124  0.683
tilde         lift             0.556  0.127  0.085  0.640
superpoint    sift             0.523  0.037  0.458  0.601
lift          sift             0.520  0.132  0.111  0.685
superpoint    tfeat            0.512  0.036  0.426  0.578
              superpoint       0.478  0.094  0.304  0.632
tilde         sift             0.457  0.128  0.063  0.638
              tfeat            0.429  0.104  0.063  0.559
lift          tfeat            0.423  0.109  0.079  0.649
sift          lift             0.406  0.243  0.008  0.652
              sift             0.401  0.250  0.007  0.700
superpoint    doap             0.376  0.028  0.315  0.429
sift          tfeat            0.374  0.231  0.007  0.690
lift          doap             0.333  0.057  0.089  0.385
tilde         doap             0.323  0.074  0.050  0.405
tcovdet       lift             0.322  0.217  0.054  0.669
              tfeat            0.227  0.154  0.036  0.531
              sift             0.212  0.153  0.000  0.487
              doap             0.189  0.130  0.029  0.411
sift          doap             0.180  0.167  0.000  0.406

In [88]:
# Die beste Detektor/Deskriptor Kombination lautet bestDD
bestDD = df_by_ddcombi.iloc[0].name
print('Die Detektor/Deskriptor Kombination ist {}.'.format(bestDD))

Die Detektor/Deskriptor Kombination ist ('superpoint', 'lift').


In [89]:
# Bewertung der Detektoren unabhängig vom Deskriptor
df_by_ddcombi.groupby('detector_name') \
    .mean() \
    .sort_values(by=['mean'], ascending=False) \
    .round(3)

,mean,std,min,max
detector_name,,,,
superpoint,0.497,0.043,0.412,0.573
lift,0.463,0.101,0.101,0.601
tilde,0.441,0.108,0.065,0.561
sift,0.340,0.223,0.005,0.612
tcovdet,0.237,0.164,0.030,0.524


In [91]:
# Bewertung der Deskriptoren unabhängig vom Detektor

df_by_ddcombi.groupby('descriptor_name') \
    .mean() \
    .sort_values(by=['mean'], ascending=False) \
    .round(3)

,mean,std,min,max
descriptor_name,,,,
lift,0.491,0.143,0.166,0.653
superpoint,0.478,0.094,0.304,0.632
sift,0.423,0.140,0.128,0.622
tfeat,0.393,0.127,0.122,0.601
doap,0.280,0.091,0.097,0.407


Wir untersuchen im Folgendene nur noch die folgenden Detektor/Deskriptor Kombinationen:
 - 1) SuperPoint/LIFT
 - 2) LIFT/LIFT
 - 3) SuperPoint/SuperPoint
 - 4) SIFT/SIFT